In [1]:
from socket import *
from threading import *
import os,stat
import trainLib as lib
import pickle
import sqlite3
import time

# allgrids = {}
# allgridslock = Lock()

# gridlocks = {} # dict: key: gridname value: lock
# gridlockslock = Lock()

gridobservers = {}
obslock = Lock()
# add grid when its created
# key: gridname, value: list of GridObservers 
class GridObserver():
    def __init__(self, sock: socket):
        self.sock = sock
        return
    def notify(self, state):
        # send updated grid.view
        print('observer received:', state)  
        for row in state:
            print(row)
        self.sock.send(state.encode())
        print('observer sent received state to client')
        return

class GridWrapper():
    def __init__(self, grid, gridname):
        self.grid = grid
        self.gridname = gridname
        self.userlist = []
        self.isShared = False

    # def createlock(self):
    #     global gridlocks
    #     with gridlockslock:
    #         gridlocks[self.gridname] = Lock()

def simulator():
    con = sqlite3.connect('trainsim.db')
    cursor = con.cursor()
    timestep = 1
    while(1):
        # load all Grids from DB
        # Run sim for all of them.
        # notify their observers
        print("simulating one frame")
        query = "SELECT * FROM grid"
        cursor.execute(query)
        result = cursor.fetchall()
        for griddata in result:
            gridname = griddata[0]
            data = griddata[1]
            grid = pickle.loads(data)

            if(grid.isRunning):
                print("advance sim on grid, from simualtor")
                grid.advanceSim()
                with obslock:
                    itsobservers = gridobservers[gridname]
                    for observer in itsobservers:
                        print("notify observer?")
                        observer.notify(grid.getdisplayable())
                pickled  = pickle.dumps(grid)
                query = "UPDATE grid SET data = ? WHERE gridname = ?"
                cursor.execute(query,(pickled,gridname))
            else:
                print("grid is not running: " + gridname)
        con.commit()
        time.sleep(timestep)
    return
def worker(sock : socket):
    
    global allgridslock, allgrids, gridlocks, gridlockslock
    global gridobservers
    # serves client with username user1
    # each worker should now the username of the connected client. username will be used in lib calls

    username = "user1"
    password = "default"
    myobserver = GridObserver(sock)
    con = sqlite3.connect('trainsim.db')

    # later, this is modified as the result of attach gridname command
    attachedGridName = "default"

    while True:
        error = 'ERROR'.encode()
        space = ' '.encode()
        msg = sock.recv(4096)
        # parse msg
        #m = msg.decode()
        #m = m + " received."
        args = msg.split()
        command = args[0]
        command = command.decode()
        if (True):
            if command == 'create': # CREATE row col gridname
                msg = 'GRID CREATED'.encode()
                row = int(args[1].decode())
                col = int(args[2].decode())
                gridname = args[3].decode()
                #func call
                creategrid(row,col,gridname, con)

                with obslock:
                    gridobservers[gridname] = [] # empty list of observers for now.

                sock.send(args[1]+ 'x'.encode()+args[2]+space+msg)

            elif command == "login": # LOGIN username password                
                username = args[1].decode()
                password = args[2].decode()

                sock.send("Hello ".encode() + args[1])


            elif command == 'attach':
                # ATTACH gridname
                gridname = args[1].decode()

                attachedGridName = gridname
                with obslock:
                    gridobservers[attachedGridName].append(myobserver)
                    print("attached " + username + " to observer list.")

                cursor = con.cursor()
                query = "UPDATE user SET gridname = ? WHERE username = ?"
                cursor.execute(query, (attachedGridName, username))
                con.commit()
                sock.send('Attached to grid: '.encode() + args[1])

            elif command == 'add': #add type row col
                type = args[1].decode()
                row = int(args[2].decode())
                col = int(args[3].decode())
                msg = 'TYPE ELEMENT ADDED AT'.encode()

                cursor = con.cursor()
                query = "SELECT data FROM grid WHERE gridname = ?"
                cursor.execute(query,(attachedGridName,))
                result = cursor.fetchone()
                grid = pickle.loads(result[0])

                grid.addElement(getnewelem(type, grid), row, col)

                pickled = pickle.dumps(grid)

                query = "UPDATE grid SET data = ? WHERE gridname = ?"
                cursor.execute(query, (pickled, attachedGridName))
                con.commit()
                
                # Notify client
                sock.send(grid.getdisplayable().encode())

            elif command == 'remove': #remove row col
                row = int(args[1].decode())
                col = int(args[2].decode())
                msg = 'ELEMENT REMOVED AT'.encode()
                sock.send(msg + space + args[1] + space + args[2])

            elif command == 'chstate': #chstate row col
                row = int(args[1].decode())
                col = int(args[2].decode())
                msg = 'STATE OF THE SWITCH AT'.encode()
                msg2 = 'CHANGED'.encode
                sock.send(msg + space + args[1] + space + args[2] + space + msg2)
            
            elif command == 'rotate':  #rotate times row col
                times = int(args[1].decode())
                row = int(args[2].decode())
                col = int(args[3].decode())
                msg = 'ELEMENT AT'.encode()
                msg2 = 'ROTATED'.encode
                msg3 = 'TIMES'.encode()
                sock.send(msg + space + args[2] + space + args[3] + space + msg2 + space + args[1] + msg3)

            elif command == 'getdur':  #getdur row col dir
                row = int(args[1].decode())
                col = int(args[2].decode())
                dir = args[3].decode()
                msg = 'DURATION FOR CELL '.encode()
                sock.send(msg + space + args[1] + space + args[2] +':'.encode()) ##add result

            elif command == 'getnext':  #getnext row col dir
                row = int(args[1].decode())
                col = int(args[2].decode())
                dir = args[3].decode()
                msg = 'NEXTCELL FOR CELL '.encode()
                sock.send(msg + space + args[1] + space + args[2] +':'.encode()) ##add result
            
            elif command == 'GETSTATUS':  #getstatus row col
                row = int(args[1].decode())
                col = int(args[2].decode()) 
                msg = 'STATUS OF CELL '.encode()
                sock.send(msg + space + args[1] + space + args[2] +':'.encode()) ##add result

            elif command == 'GETSTOP':  #getstop row col dir
                row = int(args[1].decode())
                col = int(args[2].decode())
                dir = args[3].decode()
                msg = 'STOP FOR CELL '.encode()
                sock.send(msg + space + args[1] + space + args[2] +':'.encode()) ##add result 
            
            elif command == 'start':  #startsim
                msg='SIMULATION STARTED'.encode()

                cursor = con.cursor()
                query = "SELECT data FROM grid WHERE gridname = ?"
                cursor.execute(query,(attachedGridName,))
                res = cursor.fetchone()
                grid = pickle.loads(res[0])
                grid.startSimulation()

                pickled = pickle.dumps(grid)

                query = "UPDATE grid SET data = ? WHERE gridname = ?"
                cursor.execute(query,(pickled, attachedGridName))
                con.commit()
                sock.send(msg)

            elif command == 'stop':
                msg='SIMULATION STOPPED'.encode()
                sock.send(msg)

            elif command == 'pasue':
                msg='SIMULATION PAUSED'.encode()
                sock.send(msg)

            elif command == 'STOPDSP': 
                msg='DISPLAYING STOPPED'.encode()
                sock.send(msg)

            elif command == 'bye':
                msg = 'BYE TO YOU'.encode()
                sock.send(msg)
                break

            else:
                sock.send(error)
        # some other shit

        # with allgridslock:
        #     allgrids[gridname] = wrappedgrid

        #if command ADD 

        #sock.send(m.encode())
        #sock.send(command)


def getnewelem(typeStr, globalGrid):
    if(typeStr == "regular"):
        newElm = lib.RegularRoad(True, globalGrid)
    elif(typeStr == "switch1"):
        newElm = lib.SwitchRoad(1, globalGrid)
    elif(typeStr == "switch2"):
        newElm = lib.SwitchRoad(2, globalGrid)
    elif(typeStr == "switch3"):
        newElm = lib.SwitchRoad(3, globalGrid)
    elif(typeStr == "bridge"):
        newElm = lib.BridgeCrossing(globalGrid)
    elif(typeStr == "levelcrossing"):
        newElm= lib.LevelCrossing(globalGrid)
    elif(typeStr == "leftturn"):
        newElm = lib.RegularRoad(False, globalGrid)
        newElm.makeLeftTurn()
    elif(typeStr == "rightturn"):
        newElm = lib.RegularRoad(False,globalGrid)
    elif(typeStr == "station"):
        newElm = lib.Station(globalGrid)
    elif(typeStr == "trainshed"):
        newElm = lib.TrainShed(globalGrid)
    else:
        print("typeOfCell(string) argument is invalid. Abort.")
        return
    return newElm

def creategrid(row, col , gridname, con):

    # global allgridslock, allgrids
    cursor = con.cursor()
    rawgrid = lib.GameGrid(row, col)
    pickledgrid = pickle.dumps(rawgrid)

    query = "INSERT INTO grid (gridname,data) VALUES (?, ?)"
    cursor.execute(query,(gridname,pickledgrid))
    con.commit()
    return

def startsim(gridname):
    global allgridslock
    # with allgridslock:
        # allgrids[gridname].


def createtableuser(con):
    
    cursor = con.cursor()
    query = "DROP TABLE IF EXISTS user"
    cursor.execute(query)
    con.commit()

    query = "CREATE TABLE user(username VARCHAR PRIMARY KEY, password VARCHAR, gridname VARCHAR, FOREIGN KEY(gridname) REFERENCES grid (gridname))"
    cursor.execute(query)
    con.commit()

def createtablegrid(con):
    
    cursor = con.cursor()
    query = "DROP TABLE IF EXISTS grid"
    cursor.execute(query)
    con.commit()

    query = "CREATE TABLE grid(gridname VARCHAR PRIMARY KEY, data VARCHAR)"
    cursor.execute(query)
    con.commit()


def server(ip, port):
    s = socket(AF_INET, SOCK_STREAM)

    
    con = sqlite3.connect('trainsim.db')
    cursor = con.cursor()
    
    # Create empty tables
    createtableuser(con)
    createtablegrid(con)
    # Create two default users
    query = "INSERT INTO user (username,password,gridname) VALUES (?, ?, ?)"
    cursor.execute(query,("doruk","123",None))
    query = "INSERT INTO user (username,password,gridname) VALUES (?, ?, ?)"
    cursor.execute(query,("hatice","456",None))
    con.commit()

    s.bind((ip,port))
    s.listen(10)    # 1 is queue size for "not yet accept()'ed connections"

    sim = Thread(target = simulator, args=())
    sim.start()

    try:
        while True:
        #for i in range(5):    # just limit # of accepts for Thread to exit
            ns, peer = s.accept()
            print(peer, "connected")
                # create a thread with new socket
            t = Thread(target = worker, args=(ns,))
            t.start()
            # now main thread ready to accept next connection
    finally:
        s.close()


if __name__ == '__main__':
	server('127.0.0.1', 34377)





